In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 13:06:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Load the taxi_zone CSV file into a DataFrame
df_zone = spark.read.csv("taxi_zone_lookup.csv", header=True)

In [6]:
# Show the first few records
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [9]:
# Create a temporary view in Spark
df_zone.createOrReplaceTempView("taxi_zone_lookup")

In [11]:
# Check the temp view
spark.sql("SELECT * FROM taxi_zone_lookup LIMIT 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [13]:
# Load the Yellow data CSV file into the dataframe
df_yellow = spark.read.csv("yellow_tripdata_2024-10.csv", header=True)

In [15]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|            1.0|          3.0|       1.0|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [17]:
# Create a temporary view in Spark
df_yellow.createOrReplaceTempView("yellow_tripdata_2024_10")

In [19]:
# Join the yellow taxi trip data with the taxi zone data based on PULocationID and LocationID
joined_df = df_yellow.join(df_zone, df_yellow.PULocationID == df_zone.LocationID)

In [21]:
# Group by 'Zone' and count the number of pickups for each zone
zone_counts = joined_df.groupBy("Zone").count()

In [23]:
# Find the zone with the minimum count (least frequent pickup zone)
least_frequent_zone = zone_counts.orderBy("count").limit(1)

In [25]:
# Show the least frequent pickup location zone
least_frequent_zone.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+

